In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

In [ ]:
import urllib
import pyaudio
import wave

# Lire des données depuis un fichier son

In [ ]:
url = 'http://www.nch.com.au/acm/11k16bitpcm.wav'
tmp_file,_ = urllib.urlretrieve (url)
tmp_file

In [ ]:
wf = wave.open(tmp_file, 'rb')
wf

In [ ]:
p = pyaudio.PyAudio()
stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

data = wf.readframes(-1)
stream.write(data)

stream.stop_stream()
stream.close()


In [ ]:
print(type(data),len(data))
npdata = np.fromstring(data, dtype=np.int16)

In [ ]:
npdata.shape

In [ ]:
plt.figure(figsize=[20,10])
plt.subplot(3,1,1)
plt.plot(npdata)
plt.subplot(3,1,2)
plt.plot(npdata[:20000])
plt.subplot(3,1,3)
plt.plot(npdata[:4000])


In [ ]:
stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

interval = {'thank':[500,2000],'you':[2000,3500]}

def out(stream,npdata):
        stream.write(npdata.tostring())

In [ ]:
word = 'thank'
out(stream,npdata[interval[word][0]:interval[word][1]])

In [ ]:
word = 'you'
out(stream,npdata[interval[word][0]:interval[word][1]])

# split

In [ ]:
from scipy.signal import convolve

In [ ]:
# filtrage par convolution
w = 100
lp = convolve(np.abs(npdata),np.ones(w)/w,mode='same')

# détection des 'blancs'
th = lp<3000

plt.figure(figsize=[20,10])
plt.plot(npdata[:4000],label='input')
plt.plot(lp[:4000],label='filtered')
plt.plot(th[:4000]*10000,label='blank')
plt.legend();

# silencing

In [ ]:
silenced = npdata.copy()
silenced[th] = 0

plt.figure(figsize=[20,10])
plt.plot(npdata[:4000],label='input')
plt.plot(silenced[:4000],label='silenced')
plt.legend();

In [ ]:
out(stream,silenced)

# split words

In [ ]:
# milieu des intervals
th[0] = 0 # force le début du premier blanc
dth = np.diff(th*1)

i_up = np.where(dth>0)[0]
i_down = np.where(dth<0)[0]

mid_blanks = [(start+stop)/2 for start,stop in zip(i_up,i_down)]

plt.plot(dth[:4000])
plt.gca().set_ylim(-1.1,1.1)
plt.gca().set_xlim(-100)
ax = plt.gca()
ax.vlines(mid_blanks, 0, 1, transform=ax.get_xaxis_transform(), colors='r')

In [ ]:
words = {}
for i,(s0,s1) in enumerate(zip(mid_blanks,mid_blanks[1:])):
    words['w_%02d'%i] = {'s':(s0,s1),'data':npdata[s0:s1],'duration':s1-s0}


# mot le plus long

In [ ]:
l_k,l_v = sorted(words.items(),key=lambda x:x[1]['duration'],reverse=True)[0]

In [ ]:
out(stream,l_v['data'])

# reverse !

In [ ]:
out(stream,l_v['data'][-1::-1])

# texte aléatoire

In [ ]:
# distribution des tailles
duration = [v['duration'] for v in words.itervalues()]
plt.hist(duration,bins=range(0,3000,100));

In [ ]:
from time import sleep
for i in range(10):
    random_k = random.choice(words.keys())
    d = words[random_k]['duration']
    while d<1500:
        random_k = random.choice(words.keys())
        d = words[random_k]['duration']
        
    print(random_k)
    out(stream,words[random_k]['data'])
        
    

# écho

In [ ]:
echo = npdata.copy()
delay = 4000
echo = np.pad(echo,delay*2,mode='constant',constant_values = 0)
for i in range(echo.size-delay):
    echo[i+delay-1] += echo[i]/3. 

In [ ]:
out(stream,echo)

# Thank you

In [ ]:
v = np.append(words['w_00']['data'],words['w_01']['data'])
out(stream,v)

In [ ]:
stream.stop_stream()
stream.close()